<a href="https://colab.research.google.com/github/DarkLord-13/Machine-Learning-01/blob/main/NLP_QA_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention, LayerNormalization, Concatenate

# Sample data
text_data_arr = [
    '''
        employee: What are the company's policies on breaks and lunches?
        hr: Employees are entitled to a 30-minute lunch break and two 15-minute breaks during the workday, as per company policy.

        employee: How are performance evaluations conducted?
        hr: Performance evaluations are conducted annually by your manager. Feedback sessions are scheduled to discuss strengths, areas for improvement, and goal-setting.

        employee: What is the process for requesting time off for personal reasons?
        hr: Employees can request time off for personal reasons through the HR portal or by informing their manager directly. Approval is subject to company policy and workload.

        employee: Does the company offer tuition reimbursement for continuing education?
        hr: Yes, the company offers tuition reimbursement for approved courses relevant to your job role or career development. Details can be found in the employee handbook.

        employee: What are the procedures for onboarding new employees?
        hr: New employees will undergo orientation sessions and training during their first week. HR will provide necessary paperwork and assist with setting up access to company systems.

        employee: How does the company handle complaints or grievances from employees?
        hr: Employees are encouraged to report complaints or grievances to HR or their manager. The company has a formal process for investigation and resolution.

        employee: What is the protocol for resigning from my position?
        hr: Employees should submit a formal resignation letter to their manager and HR. Notice period and exit procedures will be outlined in the employee handbook.

        employee: Can employees work remotely from another country?
        hr: Remote work from another country is subject to legal and tax considerations. Employees must discuss such arrangements with HR and comply with local regulations.

        employee: Are there opportunities for professional development and training outside of the company?
        hr: Yes, employees can explore external training and development opportunities with approval from their manager and HR. Reimbursement may be available for certain courses or certifications.

        employee: How does the company promote diversity and inclusion in the workplace?
        hr: The company has diversity and inclusion initiatives in place, including training programs, resource groups, and recruitment strategies aimed at fostering a diverse workforce.

        employee: What are the procedures for requesting a change in work schedule or location?
        hr: Employees should discuss proposed changes in work schedule or location with their manager. Formal requests can be submitted to HR for review and approval.

        employee: Does the company offer any programs or incentives for employee referrals?
        hr: Yes, the company offers referral bonuses for employees who refer qualified candidates who are hired and successfully onboarded.

        employee: How does the company handle promotions and career advancement?
        hr: Promotions and career advancement opportunities are based on merit, performance, and business needs. Employees can discuss their career goals with their manager and HR for guidance.

        employee: Is there a policy regarding social media usage and online presence?
        hr: Employees are expected to maintain professional conduct on social media and online platforms. The company may have guidelines or policies regarding appropriate usage and representation.

        employee: What is the procedure for requesting accommodations for disabilities?
        hr: Employees requiring accommodations for disabilities should contact HR to initiate the interactive process. The company will work to provide reasonable accommodations as required by law.


        employee: What are the working hours?
        hr: The standard working hours are from 9 AM to 6 PM, Monday to Friday

        employee: What is the dress code
        hr: The dress code is business casual from Monday to Thursday, and casual on Fridays

        employee: How do I apply for leave?
        hr: You can apply for leave through the company's HR portal. Ensure to get it approved by your manager.

        employee: What benefits are provided?
        hr: The company provides health insurance, retirement benefits, and performance bonuses.

        employee: How can I get my employee ID?
        hr: You will receive your employee ID on your first day from the HR department.

        employee: What is the company's policy on remote work?
        hr: The company offers flexible remote work options, but it must be approved by your manager.

        employee: How do I access the company's intranet?
        hr: You can access the company's intranet using your employee credentials through the internal network.

        employee: What should I do if I have a problem with my payroll?
        hr: If you have a payroll issue, contact the payroll department via the HR portal or email payroll@company.com.

        employee: Are there any employee training programs?
        hr: Yes, the company offers various training programs for skill development and career advancement. Check the HR portal for more details.

        employee: How can I update my personal information?
        hr: You can update your personal information through the HR portal under the 'Profile' section.

        employee: What is the procedure for reporting harassment?
        hr: To report harassment, contact the HR department directly or use the anonymous reporting feature on the HR portal.

        employee: How do I find out about job openings within the company?
        hr: Internal job openings are posted on the company's intranet under the 'Careers' section.

        employee: What is the company's travel policy?
        hr: The travel policy includes guidelines for booking travel, expense reimbursement, and travel safety. Details can be found on the HR portal.

        employee: How can I contact the IT support team?
        hr: You can contact IT support via the helpdesk portal or by calling the IT support hotline at 123-456-7890.

        employee: What is the company's policy on overtime?
        hr: Overtime must be pre-approved by your manager. Overtime pay is calculated according to state and federal laws.

        employee: How can I participate in the company's wellness programs?
        hr: Information on wellness programs, including how to sign up, is available on the HR portal under 'Wellness Programs'.

        employee: What resources are available for mental health support?
        hr: The company offers mental health support through Employee Assistance Programs (EAP). Contact HR for more information.

        employee: What is the process for getting reimbursed for work-related expenses?
        hr: Submit your receipts and expense report through the expense management system on the HR portal.

        employee: How can I access my pay stubs?
        hr: Pay stubs are available online through the payroll section of the HR portal.

        employee: What are the company's holiday policies?
        hr: The company observes all major national holidays. A detailed list of holidays is available on the HR portal.

        employee: What benefits does the company offer for parental leave?
        hr: The company provides paid parental leave for both mothers and fathers, in accordance with local laws and company policy.

        employee: How does the company handle employee grievances related to discrimination or harassment?
        hr: Employees can report incidents of discrimination or harassment to HR or a designated hotline. Investigations will be conducted promptly and confidentially.

        employee: Are there opportunities for flexible work arrangements, such as telecommuting or compressed workweeks?
        hr: Yes, the company offers flexible work arrangements based on job role and business needs. Employees can discuss options with their manager and HR.

        employee: What wellness programs does the company offer to support employee health?
        hr: The company provides wellness programs, including gym memberships, mental health resources, and health screenings, to promote employee well-being.

        employee: How does the company support employees' career growth and advancement within the organization?
        hr: Career development programs, mentorship opportunities, and internal job postings are available to support employees' professional growth and advancement.

        employee: What measures does the company have in place to ensure workplace safety?
        hr: Workplace safety protocols, including regular inspections, safety training, and emergency procedures, are implemented to maintain a safe work environment for all employees.

        employee: Does the company offer assistance with relocation for employees transferring to different locations?
        hr: Yes, relocation assistance, including moving expenses and temporary housing, may be provided for employees relocating for job assignments or transfers.

        employee: How does the company promote work-life balance for its employees?
        hr: Work-life balance initiatives, such as flexible scheduling, remote work options, and wellness programs, are promoted to support employees' personal and professional well-being.

        employee: What are the procedures for reporting a conflict of interest?
        hr: Employees should disclose any potential conflicts of interest to HR or their manager. The company has policies and procedures in place to address conflicts ethically and transparently.

        employee: Does the company offer financial assistance for employees pursuing further education or certifications?
        hr: Financial assistance, such as tuition reimbursement or scholarships, may be available for employees seeking to further their education or obtain professional certifications relevant to their role.")

    '''
]

# Preprocess the text data
text_data = text_data_arr[0].strip().split  \n\n")
questions = []
answers = []

for qa_pair in text_data:
    qa_lines = qa_pair.strip().split  \n")
    if len(qa_lines) == 2:
        questions.append(qa_lines[0].replace  employee: ", "").strip())
        answers.append(qa_lines[1].replace  hr: ", "").strip())

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Pad sequences
max_sequence_length = max(max(len(seq) for seq in question_sequences), max(len(seq) for seq in answer_sequences))
question_sequences = pad_sequences(question_sequences, maxlen=max_sequence_length, padding='post')
answer_sequences = pad_sequences(answer_sequences, maxlen=max_sequence_length, padding='post')

# One-hot encode the labels
answer_sequences = tf.keras.utils.to_categorical(answer_sequences, num_classes=vocab_size)


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 21)]                 0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, 21, 64)               9920      ['input_5[0][0]']             
                                                                                                  
 bidirectional_10 (Bidirect  (None, 21, 256)              197632    ['embedding_6[0][0]']         
 ional)                                                                                           
                                                                                                  
 layer_normalization_5 (Lay  (None, 21, 256)              512       ['bidirectional_10[0][0]

In [ ]:
from keras.layers import Input, Embedding, LSTM, Bidirectional, LayerNormalization, Attention, Concatenate, Dense
from keras.models import Model
from keras.optimizers import Adam

# Define the model
input_layer = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length)(input_layer)

# LSTM layers
lstm_layer1 = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer='l2'))(embedding_layer)
lstm_layer2 = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer='l2'))(lstm_layer1)
lstm_layer3 = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer='l2'))(lstm_layer2)

# Layer Normalization
normalized_layer = LayerNormalization()(lstm_layer3)

# Attention mechanism
query = normalized_layer
value = normalized_layer
attention_layer = Attention()([query, value])

# Concatenate attention output with LSTM output
concat_layer = Concatenate()([attention_layer, normalized_layer])

# Dense layer for additional non-linearity
dense_layer = Dense(512, activation='relu', kernel_regularizer='l2')(concat_layer)

# Output layer
output_layer = Dense(vocab_size, activation='softmax')(dense_layer)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Model summary
model.summary()


In [23]:
# Train the model
epochs = 1000
batch_size = 32

model.fit(question_sequences, answer_sequences, epochs=epochs, batch_size=batch_size)

Epoch 1/1000
1/1 [==============================] - 16s 16s/step - loss: 5.0342 - accuracy: 0.0030
Epoch 2/1000
1/1 [==============================] - 0s 200ms/step - loss: 4.6965 - accuracy: 0.2173
Epoch 3/1000
1/1 [==============================] - 0s 206ms/step - loss: 4.4293 - accuracy: 0.2173
Epoch 4/1000
1/1 [==============================] - 0s 217ms/step - loss: 4.2329 - accuracy: 0.2173
Epoch 5/1000
1/1 [==============================] - 0s 202ms/step - loss: 4.1837 - accuracy: 0.2173
Epoch 6/1000
1/1 [==============================] - 0s 203ms/step - loss: 4.0960 - accuracy: 0.2292
Epoch 7/1000
1/1 [==============================] - 0s 201ms/step - loss: 3.9771 - accuracy: 0.2619
Epoch 8/1000
1/1 [==============================] - 0s 211ms/step - loss: 3.9417 - accuracy: 0.2738
Epoch 9/1000
1/1 [==============================] - 0s 225ms/step - loss: 3.9078 - accuracy: 0.2708
Epoch 10/1000
1/1 [==============================] - 0s 203ms/step - loss: 3.8536 - accuracy: 0.2708


In [24]:
model.save  hr_lstm_model.h5")
model = tf.keras.models.load_model  hr_lstm_model.h5")

In [25]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Tokenizer should be the same as used during training
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(questions + answers)
# Ensure the tokenizer is the same instance as used during training

def preprocess_input(input_text, tokenizer, max_sequence_length):
    # Tokenize and pad the input text
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')
    return input_seq

def predict_response(input_text):
    # Preprocess the input text
    input_seq = preprocess_input(input_text, tokenizer, max_sequence_length)

    # Predict the response
    predictions = model.predict(input_seq)

    # Convert predictions to text
    predicted_sequence = np.argmax(predictions, axis=-1)[0]

    # Decode the predicted sequence
    response = ' '.join([tokenizer.index_word.get(idx, '') for idx in predicted_sequence if idx != 0])

    return response

# Example usage
input_text = "What is the company's dress code?
response = predict_response(input_text)
print  Response:", response)

1/1 [==============================] - 1s 1s/step
Response: travel policy guidelines for booking travel expense reimbursement and travel safety details can can on the the hr portal


In [26]:
input_text = "What is the company's working hours"
response = predict_response(input_text)
print  Response:", response)

1/1 [==============================] - 0s 36ms/step
Response: travel policy guidelines for booking travel expense reimbursement and travel safety details can can on the the hr portal
